In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:

##? 1st - left, 2nd - right, 3rd - up, 4th - down
##? (x, y) - order

SquareVersors = np.array([[-1, 0], [1, 0], [0, 1], [0, -1]], dtype=np.int32)

##! F points

arrFC = np.array([1, 0, 0, 0], dtype=np.int32)
arrFB = np.array([0, 1, 0, 0], dtype=np.int32)
arrFE = np.array([0, 0, 0, 1], dtype=np.int32)
arrFD = np.array([0, 0, 1, 0], dtype=np.int32)


##! B points

arrBC = np.array([1, 0, 1, 1], dtype=np.int32)
arrBB = np.array([0, 1, 1, 1], dtype=np.int32)
arrBE = np.array([1, 1, 0, 1], dtype=np.int32)
arrBD = np.array([1, 1, 1, 0], dtype=np.int32)


##! V points

arrVC = np.array([1, 0, 0, 1], dtype=np.int32)
arrVB = np.array([0, 1, 1, 0], dtype=np.int32)
arrVE = np.array([0, 1, 0, 1], dtype=np.int32)
arrVD = np.array([1, 0, 1, 0], dtype=np.int32)


###! L points

arrLH = np.array([0, 0, 1, 1], dtype=np.int32)
arrLV = np.array([1, 1, 0, 0], dtype=np.int32)


In [3]:
dictStr2Arr = { 
    'FC': arrFC, 'FB': arrFB, 'FE': arrFE, 'FD': arrFD,
    'BC': arrBC, 'BB': arrBB, 'BE': arrBE, 'BD': arrBD,
    'VC': arrVC, 'VB': arrVB, 'VE': arrVE, 'VD': arrVD,
    'LH': arrLH, 'LV': arrLV
}

lookUpFunc = lambda x: dictStr2Arr.get(x, x)  # Default to x if not found in dict

In [4]:

##! improve vec comoutation: outside the loop

def getPointsSquare(x, y, arr):
    pos0 = np.array([2*x+1, 2*y+1], dtype=np.int32)
    vec = arr.reshape(-1, 1)*SquareVersors
    
    mask = np.all(vec == 0, axis=1)
    result = np.full_like(vec, -1)
    result[~mask] = pos0 + vec[~mask]

    return result


vecTot = np.array([1, 1, 1, 1], dtype=np.int32)

def getAllPointsSquare(x, y,):
    pos0 = np.array([2*x+1, 2*y+1], dtype=np.int32)
    vec = vecTot.reshape(-1, 1)*SquareVersors
    
    result = vec + pos0
    return result

In [5]:
def goalTest(tableInit):
    
    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=np.int32)
    
    dotsX2 = np.zeros((mSize-1, mSize, 2, 2), dtype=np.int32) ##! slices along x constant
    dotsY2 = np.zeros((mSize, mSize-1, 2, 2), dtype=np.int32) ##! slices along y constant
    
    dotsX2[:, :, 0, :] = dataPointsGrid[:-1, :, 1]
    dotsX2[:, :, 1, :] = dataPointsGrid[1:, :, 0]

    dotsY2[:, :, 0, :] = dataPointsGrid[:, :-1, 2]
    dotsY2[:, :, 1, :] = dataPointsGrid[:, 1:, 3]
    
    matLogicalX = (dotsX2[:, :, 0, :] == dotsX2[:, :, 1, :]) & (np.any(dotsX2[:, :, 1, :] > -1)) & (np.any(dotsX2[:, :, 0, :] > -1))
    matLogicalY = (dotsY2[:, :, 0, :] == dotsY2[:, :, 1, :]) & (np.any(dotsX2[:, :, 1, :] > -1)) & (np.any(dotsX2[:, :, 1, :] > -1))
    
    resLogicalX = np.all(matLogicalX, axis=2)
    resLogicalY = np.all(matLogicalY, axis=2)
    
    return resLogicalX, resLogicalY
    # return np.all(resLogicalX) and np.all(resLogicalY)

In [6]:
def strategy1(tableInit):
    
    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=np.int32)
    matLog1 = np.any(np.logical_or(dataPointsGrid == 0,dataPointsGrid == 2*mSize), axis=(2,3))
    idxMat = np.where(matLog1)
     
    
    def strategy1Aux(tableArr, x, y):
    
        result = getPointsSquare(x, y, tableArr)
        bool2 = np.any(np.logical_or(result == 0, result == 2*mSize))
        
        return bool2
    
    
    for x, y in zip(*idxMat):
        
        setF = ['FC', 'FB', 'FE', 'FD']
        setB = ['BC', 'BB', 'BE', 'BD']
        setV = ['VC', 'VB', 'VE', 'VD']
        setL = ['LH', 'LV']
        
        stringValue = tableInit[x, y]
        
        # print(stringValue)

        if stringValue in setF:
            bool1 = True
            while bool1:
                
                string2 = setF[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                
                setF.remove(string2)
                
            tableInit[x, y] = string2
            
        elif stringValue in setB:
            
            setB.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setB[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                setB.remove(string2)
                
            tableInit[x, y] = string2

        elif stringValue in setV:
            
            setV.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setV[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                setV.remove(string2)
                
            tableInit[x, y] = string2
                
        elif stringValue in setL:
            
            setL.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setL[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                setL.remove(string2)
                
            tableInit[x, y] = string2

In [11]:
def strategyZero(tableInit):

    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=np.int32)
    
    setF = np.array(['FC', 'FB', 'FE', 'FD'])
    setB = np.array(['BC', 'BB', 'BE', 'BD'])
    setV = np.array(['VC', 'VB', 'VE', 'VD'])
    setL = np.array(['LH', 'LV'])
    
    idxMatB = np.isin(tableInit, setB)
    idxMatL = np.isin(tableInit, setL)

    maskBorder = np.full_like(tableInit, False)
    maskBorder[0, :] = True
    maskBorder[-1, :] = True
    maskBorder[:, 0] = True
    maskBorder[:, -1] = True


    ##! obtain the external B and L pieces
    idxMatBL = np.logical_or(idxMatB, idxMatL)
    
    matDetermWrong = np.logical_and(np.any(np.logical_or(dataPointsGrid == 0,dataPointsGrid == 2*mSize), axis=(2,3)), idxMatBL)
    matDetermRight = np.logical_and(np.logical_and(idxMatBL, np.logical_not(matDetermWrong)), maskBorder)
    
    pointsDeterm = np.where(np.logical_and(idxMatBL, matDetermWrong))
    
    idxDeterm = np.array(list(zip(*pointsDeterm)))
    idGridDeterm = np.array([tableInit[x, y] for x, y in zip(*idxDeterm)])
    
    ##! putting the deterministic pices in the proper place
    for i in range(idxDeterm.shape[0]):
        x, y = idxDeterm[i]
        idPoint = tableInit[x, y]
        
        if idPoint in setB:
            
            gridPointsSet = np.array([getPointsSquare(x, y, lookUpFunc(piece)) for piece in setB], dtype=np.int32)
            checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
            
            idxCheckBoundaries = np.where(~checkBoundaries)[0]
            
            if len(idxCheckBoundaries) > 1:
                print("error: more than one possible solution")
            else:
                tableInit[x, y] = setB[idxCheckBoundaries[0]]
                tableArr[x, y] = lookUpFunc(setB[idxCheckBoundaries[0]])
            
        elif idPoint in setL:
            
            gridPointsSet = np.array([getPointsSquare(x, y, lookUpFunc(piece)) for piece in setL], dtype=np.int32)
            checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
            
            idxCheckBoundaries = np.where(~checkBoundaries)[0]
            
            if len(idxCheckBoundaries) > 1:
                print("error: more than one possible solution")
            else:
                tableInit[x,y] = setL[idxCheckBoundaries[0]]
                tableArr[x, y] = lookUpFunc(setL[idxCheckBoundaries[0]])
    
    # print(tableInit)
    
    ##! building the rest of the external layer which now is deterministic
    
    ##! Auxiliar Important Functions
    def getAdjacentPoints(x, y, maskFrameCheck):
        
        checkFrame2 = np.full_like(tableInit, False)
        xMod = x-1
        yMod = y-1
        if xMod < 0:
            xMod = 0
        if yMod < 0:
            yMod = 0
        checkFrame2[xMod:x+2, y] = True
        checkFrame2[x, yMod:y+2] = True
        
        checkFrame2 = np.logical_and(checkFrame2, maskFrameCheck)
        return np.where(checkFrame2)
    
    
    def strategyOne(x2, y2, maskFrameCheck, boolLayerExt=False):
        
        idx2 = getAdjacentPoints(x2, y2, maskFrameCheck)
        detAdjPoints = np.array([getPointsSquare(x, y, tableArr[x,y]) for x, y in zip(*idx2)], dtype=np.int32)
        
        setF = np.array(['FC', 'FB', 'FE', 'FD'])
        setB = np.array(['BC', 'BB', 'BE', 'BD'])
        setV = np.array(['VC', 'VB', 'VE', 'VD'])
        setL = np.array(['LH', 'LV'])
        
        idPoint = tableInit[x2, y2]
    
        ## get the points of the boundary
        pointsXY = getAllPointsSquare(x2, y2)
        
        ## get the points and the coordinates of the deterministic external points
        matches = np.array([np.any((detAdjPointsAux[:, np.newaxis, :] == pointsXY).all(axis=2), axis=1) for detAdjPointsAux in detAdjPoints])
        idxMatch = np.where(matches)
        idxMatchMat = np.array(list(zip(*idxMatch)))
        
        ## get the proper Match points on the boundary
        if idxMatchMat.size != 0:
            properPointsBoundary = detAdjPoints[idxMatchMat[:, 0], idxMatchMat[:, 1]]
        else:
            properPointsBoundary = np.array([], dtype=np.int32)
        
        
        ##! hipothessis of rejecting pieces that are not possible
        pointsAdjXY = np.array([getAllPointsSquare(x, y) for x, y in zip(*idx2)], dtype=np.int32)
        matchesAllPossible = np.array([np.any((pointsAdjXYAux[:, np.newaxis, :] == pointsXY).all(axis=2), axis=1) for pointsAdjXYAux in pointsAdjXY])
        idxMatchAllPossible = np.where(matchesAllPossible)
        idxMatchMatAllPossible = np.array(list(zip(*idxMatchAllPossible)))
        
        maskNotPossible = np.logical_and(matchesAllPossible, np.logical_not(matches))
        idxNotPossible = np.where(maskNotPossible)
        idxMatchMatNotPossible = np.array(list(zip(*idxNotPossible)))
        
        if idxMatchMatNotPossible.size != 0:
            properPointsBoundaryNot = pointsAdjXY[idxMatchMatNotPossible[:, 0], idxMatchMatNotPossible[:, 1]]
        else:
            properPointsBoundaryNot = np.array([], dtype=np.int32)
        
        actions = []
        
        if idPoint in setF:
            
            ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setF], dtype=np.int32)
            
            if properPointsBoundaryNot.size !=0:
                checkSimilarityNot = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundaryNot, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=np.int32)
                checkSimilarityNot = np.sum(checkSimilarityNot, axis=1) != 0
            else:
                checkSimilarityNot = np.full((gridPointsSet.shape[0],), False, dtype=np.bool_)
            
            
            ## in case of not having points on the boundary, the full set is returned and we arent in the external layer
            if properPointsBoundary.size == 0 and np.logical_not(boolLayerExt):
                return setF[np.logical_not(checkSimilarityNot)]
            
            elif properPointsBoundary.size == 0 and boolLayerExt:

                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                idxCheckBoundaries = np.logical_not(np.logical_or(checkBoundaries, checkSimilarityNot))
                
                return setF[idxCheckBoundaries]
            elif boolLayerExt:
                
                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                idxCheckBoundaries = np.logical_not(np.logical_or(checkBoundaries, checkSimilarityNot))
                gridPointsSet = gridPointsSet[idxCheckBoundaries]
            else:
                gridPointsSet = gridPointsSet[np.logical_not(checkSimilarityNot)]
            
            # ## get the points of all the possible points available
            # gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setF], dtype=np.int32)
            
            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=np.int32)
            similarityVec = np.sum(similarityVec, axis=1)
            
            ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setF[*np.where(similarityVec == np.max(similarityVec))]
            
        elif idPoint in setV:
            
            if properPointsBoundary.size == 0 and np.logical_not(boolLayerExt):
                return setV
            
            ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setV], dtype=np.int32)
            
            if properPointsBoundary.size == 0 and boolLayerExt:
                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                idxCheckBoundaries = np.where(~checkBoundaries)[0]
                return setV[idxCheckBoundaries]
            elif boolLayerExt:
                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                idxCheckBoundaries = np.where(~checkBoundaries)[0]
                gridPointsSet = gridPointsSet[idxCheckBoundaries]
            else:
                pass
            
            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=np.int32)
            similarityVec = np.sum(similarityVec, axis=1)
            
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setV[*np.where(similarityVec == np.max(similarityVec))]
            
        elif idPoint in setL:
            
            if properPointsBoundary.size == 0:
                return setL
            
            # ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setL], dtype=np.int32)

            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=np.int32)
            similarityVec = np.sum(similarityVec, axis=1)
            
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setL[*np.where(similarityVec == np.max(similarityVec))]
            
        elif idPoint in setB:
            
            ## in case of not having points on the boundary, the full set is returned
            if properPointsBoundary.size == 0:
                return setB

            # ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setB], dtype=np.int32)

            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=np.int32)
            similarityVec = np.sum(similarityVec, axis=1)
            
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setB[*np.where(similarityVec == np.max(similarityVec))]
    
        return actions
    
    ##! Build the zeroth mask Frame Check
    matDetermBoth = np.logical_or(matDetermRight, matDetermWrong)
    idxDetermBoth = np.where(matDetermBoth)
    
    matActions = np.zeros((mSize, mSize), dtype=object)
    boolNonDetermSystem = False
    
    maskFrameCheck = np.full_like(tableInit, False)
    
    
    ##! Zero Iteration
    maskAux = np.full_like(tableInit, False)
    for x, y in zip(*idxDetermBoth):
        maskFrameCheck[x, y] = True
            
        maskAux[x-1:x+2, y] = True
        maskAux[x, y-1:y+2] = True
        
    for x, y in zip(*idxDetermBoth):
        maskAux[x, y] = False
    
    maskBorderZero = np.logical_and(maskBorder, maskAux)
    
    idx2ChangeBorderZero = np.array(list(zip(*np.where(maskBorderZero))))
    idx2FrameCheck = np.array(list(zip(*np.where(maskFrameCheck))))
    
    for i in range(idx2ChangeBorderZero.shape[0]):
        x2, y2 = idx2ChangeBorderZero[i]
        actions = strategyOne(x2, y2, maskFrameCheck)
        
        # print(x2, y2, actions)
        if len(actions) == 1:
            tableInit[x2, y2] = actions[0]
            tableArr[x2, y2] = lookUpFunc(actions[0])
            maskFrameCheck[x2, y2] = True
        else:
            boolNonDetermSystem = True
            matActions[x2, y2] = actions
    
    # print(tableInit)
    # print()
    # print(matActions)
    
    ##! Recurrent Iterations
    boolLayerExt = True
    maskLoopCheck = maskFrameCheck.copy()
    
    while boolLayerExt:
        idxDeterm = np.where(maskLoopCheck)
        
        ##! Define the mask for the new boundaries
        maskAux = np.full_like(tableInit, False)
        for x, y in zip(*idxDeterm):
            maskAux[x-1:x+2, y] = True
            maskAux[x, y-1:y+2] = True
        
        for x, y in zip(*idxDeterm):
            maskAux[x, y] = False
        
        ##! ensure we do not leave the external layer
        maskBorderZero = np.logical_and(maskBorder, maskAux)
        
        idx2ChangeBorderZero = np.array(list(zip(*np.where(maskBorderZero))))
        
        ##! check if the iterattion over the external layer is finished
        if idx2ChangeBorderZero.size == 0:
            boolLayerExt = False
        
        for i in range(idx2ChangeBorderZero.shape[0]):
            x2, y2 = idx2ChangeBorderZero[i]
            actions = strategyOne(x2, y2, maskFrameCheck, boolLayerExt)
            
            # print(x2, y2, tableInit[x2, y2])
        
            if len(actions) == 1:
                tableInit[x2, y2] = actions[0]
                tableArr[x2, y2] = lookUpFunc(actions[0])
                maskFrameCheck[x2, y2] = True
            else:
                matActions[x2, y2] = actions
                
            maskLoopCheck[x2, y2] = True
    
    
    print(maskLoopCheck)
    print()
    print(tableInit)
    print(matActions)
    print(maskFrameCheck)
    
    ##! Upper Layer
    
    print("Upper Layer")
    
    # maskLevelUpper = np.array(list(zip(*np.nonzero(maskFrameCheck))))
    maskLevelUpper = np.full_like(tableInit, False)
    maskLevelUpper[1:-1, 1] = True
    maskLevelUpper[1:-1, -2] = True
    maskLevelUpper[1, 1:-1] = True
    maskLevelUpper[-2, 1:-1] = True
    
    def findBetterPoint(maskFrame):
        x2 = 0
        y2 = 0
        counterMax = 0
        idxMaskFrame = np.where(maskFrame)
        for x, y in zip(*idxMaskFrame):
            maskAuxUpper = np.full_like(tableInit, False)
            xMin = x-1
            yMax = y-1
            xMax = x+1
            yMin = y+1
            if xMin < 0:
                xMin = 0
            if yMin < 0:
                yMin = 0
            if xMax >= mSize:
                xMax = mSize-1
            if yMin >= mSize:
                yMin = mSize-1
            maskAuxUpper[xMin, y] = True
            maskAuxUpper[x, yMax] = True
            maskAuxUpper[xMax, y] = True
            maskAuxUpper[x, yMin] = True
            
            maskResult = np.logical_and(maskAuxUpper, maskFrameCheck)
            counter = np.sum(maskResult)
            
            if counter > counterMax:
                counterMax = counter
                x2 = x
                y2 = y
                
        return x2, y2, counterMax
        
        
    boolUpperLayer = True
    while boolUpperLayer:
        x2, y2, counterMax = findBetterPoint(maskLevelUpper)
        print(x2, y2, counterMax)
        
        if counterMax == 0:
            boolUpperLayer = False
        else:
            actions = strategyOne(x2, y2, maskFrameCheck, False)
            print(x2, y2, actions)
            if len(actions) == 1:
                tableInit[x2, y2] = actions[0]
                tableArr[x2, y2] = lookUpFunc(actions[0])
                maskFrameCheck[x2, y2] = True
            else:
                matActions[x2, y2] = actions
                
            maskLevelUpper[x2, y2] = False
            
    print(tableInit)
    print(matActions)
    print(maskFrameCheck)
    
    # actions = strategyOne(2, 2, maskFrameCheck, False)
    # print("2, 2: ", actions)
    
    

tableInit0 = np.array(  
    [['FC','BB','BC','FB'],
    ['FC','BD','FD','FD'],
    ['VD','BC','BC','BD'],
    ['FB','FB','VE','FD']] , dtype=object)

table4 = tableInit0.copy()


actions = strategyZero(table4)
# print(actions)
# print(table4)


[[True True True True]
 [True False False True]
 [True False False True]
 [True True True True]]

[['FD' 'BB' 'BB' 'FE']
 ['FC' 'BD' 'FD' 'FB']
 ['VD' 'BC' 'BC' 'BE']
 ['FB' 'FB' 'VE' 'FC']]
[[0 0 0 0]
 [array(['FB', 'FD'], dtype='<U2') 0 0 0]
 [array(['VB', 'VD'], dtype='<U2') 0 0 0]
 [array(['FC', 'FD'], dtype='<U2') array(['FC', 'FE', 'FD'], dtype='<U2')
  array(['VC', 'VD'], dtype='<U2') 0]]
[[True True True True]
 [False False False True]
 [False False False True]
 [False False False True]]
Upper Layer
1 2 2
1 2 ['FC']
1 1 2
1 1 ['BC' 'BE' 'BD']
2 2 2
2 2 ['BC' 'BB' 'BD']
0 0 0
[['FD' 'BB' 'BB' 'FE']
 ['FC' 'BD' 'FC' 'FB']
 ['VD' 'BC' 'BC' 'BE']
 ['FB' 'FB' 'VE' 'FC']]
[[0 0 0 0]
 [array(['FB', 'FD'], dtype='<U2') array(['BC', 'BE', 'BD'], dtype='<U2')
  0 0]
 [array(['VB', 'VD'], dtype='<U2') 0
  array(['BC', 'BB', 'BD'], dtype='<U2') 0]
 [array(['FC', 'FD'], dtype='<U2') array(['FC', 'FE', 'FD'], dtype='<U2')
  array(['VC', 'VD'], dtype='<U2') 0]]
[[True True True True]
 [False 

In [8]:
# tableInit = np.array(
#     [['FB', 'VD', 'VD'] \
#     ,['BC', 'BB', 'LV'] \
#     ,['FB', 'FB', 'FB']] , dtype=object)


# tableInit2 = np.array(
#     [['VB', 'VD'], \
#      ['FC', 'FD']], dtype=object)


# table2 = tableInit.copy()
# table3 = tableInit2.copy()

In [9]:
import numpy as np

def shift(matrix, direction):
    if direction == 'up':
        return np.vstack((matrix[1:], np.zeros(matrix.shape[1])))
    elif direction == 'down':
        return np.vstack((np.zeros(matrix.shape[1]), matrix[:-1]))
    elif direction == 'left':
        return np.hstack((matrix[:, 1:], np.zeros((matrix.shape[0], 1))))
    elif direction == 'right':
        return np.hstack((np.zeros((matrix.shape[0], 1)), matrix[:, :-1]))

# Assuming 'matrix' is your numpy array
matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Get the adjacent entries
up = shift(matrix, 'up')
down = shift(matrix, 'down')
left = shift(matrix, 'left')
right = shift(matrix, 'right')

print("Up:\n", up)
print("Down:\n", down)
print("Left:\n", left)
print("Right:\n", right)

Up:
 [[4. 5. 6.]
 [7. 8. 9.]
 [0. 0. 0.]]
Down:
 [[0. 0. 0.]
 [1. 2. 3.]
 [4. 5. 6.]]
Left:
 [[2. 3. 0.]
 [5. 6. 0.]
 [8. 9. 0.]]
Right:
 [[0. 1. 2.]
 [0. 4. 5.]
 [0. 7. 8.]]


In [10]:
# table2 = tableInit.copy()

# setF = ['FC', 'FB', 'FE', 'FD']
# setB = ['BC', 'BB', 'BE', 'BD']
# setV = ['VC', 'VB', 'VE', 'VD']
# setL = ['LH', 'LV']


# tableArr = np.array([[lookUpFunc(x) for x in row] for row in table2], dtype=np.int32)


# # print(setB)
# # stringValue = table2[x, y]
# # setB.remove(stringValue)
# # print(setB)




# x = 0
# y = 2


# stringValue = table2[x, y]

# if stringValue in setF:
#     bool1 = True
#     while bool1:
        
#         string2 = setF[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
        
#         setF.remove(string2)
        
#     table2[x, y] = string2
    
# elif stringValue in setB:
    
#     setB.remove(stringValue)
#     bool1 = True
#     while bool1:
        
#         string2 = setB[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
#         setB.remove(string2)
        
#     table2[x, y] = string2

# elif stringValue in setV:
    
#     setV.remove(stringValue)
#     bool1 = True
#     while bool1:
        
#         string2 = setV[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
#         setV.remove(string2)
        
#     table2[x, y] = string2
        
# elif stringValue in setL:
    
#     setL.remove(stringValue)
#     bool1 = True
#     while bool1:
        
#         string2 = setL[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
#         setL.remove(string2)
        
#     table2[x, y] = string2
        


# print(table2[x, y])

# while matLog2:
    
#     print(setB)
#     stringValue = table2[x, y]
#     setB.remove(stringValue)
    
#     table2[x, y] = setB[0]

#     tableArr[x, y] = lookUpFunc(setB[0])
#     result = getPointsSquare(x, y, tableArr[x, y])
#     matLog2 = np.any(np.logical_or(result == 0, result == 2*mSize))
    
    
# print(table2[x, y])